In [2]:
import pandas as pd
from datetime import datetime, timedelta, time
import pytz
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

In [3]:
df= pd.read_csv('data-NSE_FO_35000.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
df.set_index('Date', inplace=True)

In [138]:
daily_ohlc = df.resample('D').agg({
    'Open': 'first',     # First entry of the day
    'High': 'max',       # Max high of the day
    'Low': 'min',        # Min low of the day
    'Close': 'last'      # Last entry of the day
})

In [140]:
daily_ohlc.dropna(inplace=False)

,Open,High,Low,Close
Date,,,,
2024-06-28 00:00:00+05:30,24349.90,24461.75,24338.00,24363.40
2024-07-01 00:00:00+05:30,24335.40,24446.90,24315.80,24440.00
2024-07-02 00:00:00+05:30,24449.95,24509.80,24361.00,24438.95
2024-07-03 00:00:00+05:30,24509.80,24601.00,24466.70,24590.10
2024-07-04 00:00:00+05:30,24601.00,24680.00,24570.05,24605.00
2024-07-05 00:00:00+05:30,24589.25,24651.25,24487.00,24625.00
2024-07-08 00:00:00+05:30,24630.00,24640.00,24534.10,24618.00
2024-07-09 00:00:00+05:30,24618.00,24744.00,24618.00,24725.00
2024-07-10 00:00:00+05:30,24743.75,24743.75,24480.00,24599.65


In [24]:
df.index [0] == datetime(2024, 6, 28, 9, 15, tzinfo=  pytz.FixedOffset(330))

True

In [50]:
df.index.time

array([datetime.time(9, 15), datetime.time(9, 16), datetime.time(9, 17),
       ..., datetime.time(15, 27), datetime.time(15, 28),
       datetime.time(15, 29)], dtype=object)

In [49]:
pd.Series(df.index.date).unique()

array([datetime.date(2024, 6, 28), datetime.date(2024, 7, 1),
       datetime.date(2024, 7, 2), datetime.date(2024, 7, 3),
       datetime.date(2024, 7, 4), datetime.date(2024, 7, 5),
       datetime.date(2024, 7, 8), datetime.date(2024, 7, 9),
       datetime.date(2024, 7, 10), datetime.date(2024, 7, 11),
       datetime.date(2024, 7, 12), datetime.date(2024, 7, 15),
       datetime.date(2024, 7, 16), datetime.date(2024, 7, 18),
       datetime.date(2024, 7, 19), datetime.date(2024, 7, 22),
       datetime.date(2024, 7, 23), datetime.date(2024, 7, 24),
       datetime.date(2024, 7, 25), datetime.date(2024, 7, 26),
       datetime.date(2024, 7, 29), datetime.date(2024, 7, 30),
       datetime.date(2024, 7, 31), datetime.date(2024, 8, 1),
       datetime.date(2024, 8, 2), datetime.date(2024, 8, 5),
       datetime.date(2024, 8, 6), datetime.date(2024, 8, 7),
       datetime.date(2024, 8, 8), datetime.date(2024, 8, 9),
       datetime.date(2024, 8, 12), datetime.date(2024, 8, 13),
      

In [8]:
base = df.loc['2024-06-28']

In [7]:
day0 = df.loc['2024-07-01']

In [4]:
def closing(df):
  return df.Close

In [216]:



class Alphabots(Strategy):
    
    delta = 0.0007
    
    volume = 0.5

    def init(self):
      # self.base_price = self.data.Close[0]
      self.short_trade = False
      self.long_trade = False
      self.base_price = 24370.0#float((day0.Close * day0.Volume / day0.Volume.sum()).sum())
      # self.close_price = self.I(closing, self.data)
      print(self.base_price)
      
      

    def next(self):
      # print(self.data)
      # print('``````````')
      # print(self.close_price)
      if self.data.index.time[-1] == time(15, 10):
        print("hehe")
        # print(self.data.index)
        self.position.close()
      if self.data.Close[-1] > self.base_price * (1 + self.delta):
         if self.long_trade == False:
            print("dd", self.long_trade)
            # self.position.close()
            self.buy(size= 1)
            self.long_trade = True
      elif self.data.Close[-1] < self.base_price * (1 - self.delta):
         if self.short_trade == False:
            print(self.short_trade)
            # self.position.close()
            self.sell(size = 1)
            self.short_trade = True
            
      



In [217]:


bt = Backtest(day0, Alphabots, cash=200_000, commission=0.00, hedging= True)
stats = bt.run()


24370.0
False
dd False
hehe


In [218]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,27,355,24389.80,24440.1,50.30,0.002062,2024-07-01 09:43:00+05:30,2024-07-01 15:11:00+05:30,0 days 05:28:00
1,-1,2,355,24338.35,24440.1,-101.75,-0.004181,2024-07-01 09:18:00+05:30,2024-07-01 15:11:00+05:30,0 days 05:53:00


In [219]:
stats

Start                     2024-07-01 09:16...
End                       2024-07-01 15:29...
Duration                      0 days 06:13:00
Exposure Time [%]                   94.652406
Equity Final [$]                    199948.55
Equity Peak [$]                     200022.55
Return [%]                          -0.025725
Buy & Hold Return [%]                 0.39043
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.036996
Avg. Drawdown [%]                   -0.036996
Max. Drawdown Duration        0 days 06:11:00
Avg. Drawdown Duration        0 days 06:11:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       0.206234
Worst Trade [%]                     -0.418064
Avg. Trade [%]                    

In [220]:
bt.plot()

d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
d:\user\Documents\workspace\Alphabots\.venv\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\user\Documents\workspace\Alpha

GridPlot(id='p6734', ...)